In [1]:
#Imports
import pandas as pd
import numpy as np
import re
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import GaussianNB, MultinomialNB, ComplementNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV, cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.base import TransformerMixin

In [2]:
#algorithm
class todense(TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None, **fit_params):
        return self
    
    def transform(self, X, y=None, **fit_params):
        return X.toarray()


# load data
df_transactions = pd.read_csv('DataSet.csv', sep=";", header=0)

# format numbers
X = []
for num in df_transactions['Betrag']:
    X.append(num.replace(",", "."))
df_transactions['Betrag'] = X
X = 0

# set data types
df_transactions['Verwendungszweck'] = 
                        df_transactions['Verwendungszweck'].astype('str')
df_transactions['label'] = df_transactions['label'].astype("category")
verwendungszweck = df_transactions["Verwendungszweck"].tolist()

# data cleaning/ preprocessing
my_stopword_list = ["and", "to", "the", "of", "notprovided"]
rowAdapted = []
for row in verwendungszweck:
    sentence = row.lower()
    sentence = re.sub("[0-9]+", " ", sentence)
    word = re.findall(r'\w+', sentence)
    if len(sentence) > 1 and not(len(row) == 0):
        if sentence not in my_stopword_list: 
            rowAdapted.append(word)
verwendungszweck = rowAdapted

# Prediction
# MultinomialNB
clf_pipe_MNB = Pipeline([("vectorizer", TfidfVectorizer(analyzer=lambda x:x)),
                         ("clf", MultinomialNB())])
# ComplementNB
clf_pipe_CNB = Pipeline([("vectorizer", TfidfVectorizer(analyzer=lambda x:x)),
                         ("clf", ComplementNB())])
# GaussianNB
clf_pipe_GNB = Pipeline([("vectorizer", TfidfVectorizer(analyzer=lambda x:x)),
                         ("todense", todense()), ("clf", GaussianNB())])

clf_models = [clf_pipe_MNB, clf_pipe_CNB, clf_pipe_GNB]
X_train = verwendungszweck
y_train = df_transactions["label"]

for i in range(0, len(clf_models)):
    models = clf_models[i]
    if i < 2:
        parameters = {"clf__alpha": (2, 1e-2, 1e-3, 1e-4, 1e-5)}
        gridsearch_multi = GridSearchCV(models, parameters, iid= True, cv=5)
        fitter = gridsearch_multi.fit(X_train, y_train)

    else:
        parameters = {}
        gridsearch_multi = GridSearchCV(models, parameters, iid = True, cv=5)
        fitter = gridsearch_multi.fit(X_train, y_train)

    y_pred = cross_val_predict(gridsearch_multi, X_train, y_train, cv=5)
    cm = confusion_matrix(y_train, y_pred)
    cm2 = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    # printing results
    print(fitter.best_score_)
    print(cm2)

0.8803827751196173
[[0.84848485 0.         0.09090909 0.06060606 0.         0.        ]
 [0.         1.         0.         0.         0.         0.        ]
 [0.01538462 0.         0.98461538 0.         0.         0.        ]
 [0.03846154 0.         0.07692308 0.76923077 0.         0.11538462]
 [0.04761905 0.         0.23809524 0.14285714 0.57142857 0.        ]
 [0.         0.         0.08510638 0.         0.         0.91489362]]
0.8803827751196173
[[0.90909091 0.         0.03030303 0.06060606 0.         0.        ]
 [0.         1.         0.         0.         0.         0.        ]
 [0.01538462 0.         0.95384615 0.         0.         0.03076923]
 [0.03846154 0.         0.03846154 0.76923077 0.03846154 0.11538462]
 [0.28571429 0.         0.         0.14285714 0.57142857 0.        ]
 [0.         0.         0.06382979 0.         0.         0.93617021]]
0.8899521531100478
[[0.84848485 0.         0.03030303 0.06060606 0.06060606 0.        ]
 [0.         1.         0.         0.       